In [7]:
import fitsio
import numpy as np
import matplotlib.pyplot as plt
import sys, os

sys.path.append('/global/homes/q/qmxp55/DESI/BGS_paper/')
from desitarget import io
from desitarget.targets import desi_mask, bgs_mask, mws_mask

In [2]:
def flux_to_mag(flux):
    mag = 22.5 - 2.5*np.log10(flux)
    return mag

In [3]:
#nside = io.desitarget_nside()
sweep_dir_dr8south = '/global/project/projectdirs/cosmo/data/legacysurvey/dr8/south/sweep/8.0'
sweep_dir_dr8north = '/global/project/projectdirs/cosmo/data/legacysurvey/dr8/north/sweep/8.0'
#dest = '/global/cscratch1/sd/qmxp55/desitarget_output/targets-BGS-sv1-dr8_lowq_relaxed.fits'

infilessouth = io.list_sweepfiles(sweep_dir_dr8south)
infilesnorth = io.list_sweepfiles(sweep_dir_dr8north)
infiles = np.concatenate((infilessouth, infilesnorth)).tolist()

## run in master branch (before update)

In [4]:
%%bash
#!/bin/bash

cd /global/homes/q/qmxp55/DESI/desitarget
git checkout master

Your branch is up to date with 'origin/master'.


Already on 'master'


In [7]:
#
#sys.path.insert(0, '/global/homes/q/qmxp55/DESI/BGS_paper/')
sys.path.append('/global/homes/q/qmxp55/DESI/BGS_paper/')
from desitarget.cuts import select_targets
from desitarget import io
from desitarget.targets import desi_mask, bgs_mask, mws_mask
import time
start = time.time()

# setting GAIA dir environment (new requirement 5th Nov 2019)
os.environ['GAIA_DIR'] = '/global/project/projectdirs/cosmo/data/gaia/dr2/'
targets = select_targets(infiles[:2], mask=True, tcnames=["BGS"], survey='main', backup=False)

end = time.time()
print('Total run time: %f sec' %(end - start))

rmag = flux_to_mag(targets['FLUX_R']/targets['MW_TRANSMISSION_R'])
rfibmag = flux_to_mag(targets['FIBERFLUX_R']/targets['MW_TRANSMISSION_R'])
    

INFO:cuts.py:2375:select_targets: Running on the main survey
Total run time: 69.469287 sec


In [9]:
bits = ['BGS_FAINT', 'BGS_BRIGHT', 'BGS_WISE']
for bit in bits:
    bgsbit = ((targets['BGS_TARGET'] & bgs_mask[bit]) != 0)
    print(bit, np.sum(bgsbit))
    print('\t -- rmag min: %f, rmag max: %f' %(rmag[bgsbit].min(),rmag[bgsbit].max()))
    print('\t -- rfibmag min: %f, rfibmag max: %f' %(rfibmag[bgsbit].min(),rfibmag[bgsbit].max()))


BGS_FAINT 25581
	 -- rmag min: 19.500004, rmag max: 19.999994
	 -- rfibmag min: 19.772762, rfibmag max: 24.318287
BGS_BRIGHT 37201
	 -- rmag min: 11.544000, rmag max: 19.499952
	 -- rfibmag min: 15.601568, rfibmag max: 25.266115
BGS_WISE 1838
	 -- rmag min: 15.386374, rmag max: 19.999062
	 -- rfibmag min: 15.657728, rfibmag max: 20.277885


### apply maskbits and FMC cuts to master desitarget output

In [10]:
#
BS_cat = ((targets['MASKBITS'] & 2**1)==0)
LG_cat = ((targets['MASKBITS'] & 2**12)==0)
GC_cat = ((targets['MASKBITS'] & 2**13)==0)
# photometric
# Fiber Magnitude Cut
FMC = np.zeros_like(targets['RA'], dtype='?')
FMC |= ((rfibmag < (2.9 + 1.2) + rmag) & (rmag < 17.1))
FMC |= ((rfibmag < 21.2) & (rmag < 18.3) & (rmag > 17.1))
FMC |= ((rfibmag < 2.9 + rmag) & (rmag > 18.3))

maskdef_cat = (FMC) & (BS_cat) & (LG_cat) & (GC_cat)

for bit in bits:
    bgsbit = ((targets['BGS_TARGET'] & bgs_mask[bit]) != 0) & (maskdef_cat)
    print(bit, np.sum(bgsbit))
    print('\t -- rmag min: %f, rmag max: %f' %(rmag[bgsbit].min(),rmag[bgsbit].max()))
    print('\t -- rfibmag min: %f, rfibmag max: %f' %(rfibmag[bgsbit].min(),rfibmag[bgsbit].max()))


BGS_FAINT 25418
	 -- rmag min: 19.500004, rmag max: 19.999994
	 -- rfibmag min: 19.773514, rfibmag max: 22.862207
BGS_BRIGHT 35852
	 -- rmag min: 14.270964, rmag max: 19.499952
	 -- rfibmag min: 16.216063, rfibmag max: 22.376495
BGS_WISE 1827
	 -- rmag min: 15.386374, rmag max: 19.999062
	 -- rfibmag min: 15.657728, rfibmag max: 20.277885


## run in branch with updated target selection

In [4]:
%%bash
#!/bin/bash

cd /global/homes/q/qmxp55/DESI/desitarget
git checkout BGS_main


M	py/desitarget/cuts.py


Already on 'BGS_main'


In [5]:
#
sys.path.append('/global/homes/q/qmxp55/DESI/BGS_paper/')
from desitarget.cuts import select_targets
from desitarget import io
from desitarget.targets import desi_mask, bgs_mask, mws_mask

import time
start = time.time()

# setting GAIA dir environment (new requirement 5th Nov 2019)
os.environ['GAIA_DIR'] = '/global/project/projectdirs/cosmo/data/gaia/dr2/'
targets = select_targets(infiles[:2], mask=True, tcnames=["BGS"], survey='main', backup=False)

end = time.time()
print('Total run time: %f sec' %(end - start))

rmag = flux_to_mag(targets['FLUX_R']/targets['MW_TRANSMISSION_R'])
rfibmag = flux_to_mag(targets['FIBERFLUX_R']/targets['MW_TRANSMISSION_R'])
    

INFO:cuts.py:2394:select_targets: Running on the main survey
Total run time: 64.267602 sec


In [6]:
bits = ['BGS_FAINT', 'BGS_BRIGHT', 'BGS_WISE']
for bit in bits:
    bgsbit = ((targets['BGS_TARGET'] & bgs_mask[bit]) != 0)
    print(bit, np.sum(bgsbit))
    print('\t -- rmag min: %f, rmag max: %f' %(rmag[bgsbit].min(),rmag[bgsbit].max()))
    print('\t -- rfibmag min: %f, rfibmag max: %f' %(rfibmag[bgsbit].min(),rfibmag[bgsbit].max()))


BGS_FAINT 25418
	 -- rmag min: 19.500004, rmag max: 19.999994
	 -- rfibmag min: 19.773514, rfibmag max: 22.862207
BGS_BRIGHT 35852
	 -- rmag min: 14.270964, rmag max: 19.499952
	 -- rfibmag min: 16.216063, rfibmag max: 22.376495
BGS_WISE 1827
	 -- rmag min: 15.386374, rmag max: 19.999062
	 -- rfibmag min: 15.657728, rfibmag max: 20.277885
